In [1]:
#-----------------------------------------------------
# SQL_jupyterlab to create databases
# next update :
# => add function when selecting df.index
# => put all functions in a file.py
# => write a script to populate tables automatically
# => add sys.agrv for csv_file_call
#-----------------------------------------------------

import pandas as pd

# load data and drop duplicates
df = pd.read_excel("immobilier.xlsx")
df = df.drop_duplicates()

# formatting columns
df.columns = [i.replace(' ', '_').lower() for i in df.columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34169 entries, 0 to 34168
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date_mutation              34169 non-null  datetime64[ns]
 1   nature_mutation            34169 non-null  object        
 2   valeur_fonciere            34151 non-null  float64       
 3   no_voie                    34036 non-null  float64       
 4   b/t/q                      2174 non-null   object        
 5   code_type_de_voie          34169 non-null  int64         
 6   type_de_voie               33229 non-null  object        
 7   code_voie                  34169 non-null  object        
 8   voie                       34169 non-null  object        
 9   code_id_commune            34169 non-null  int64         
 10  code_postal                34168 non-null  float64       
 11  commune                    34169 non-null  object        
 12  code

C:\ProgramData\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
# select manually all columns by index required in vente_table
vente_columns = list(df.columns[:3])

# select manually all columns by index required in adresse_table
adresse_columns = list(df.columns[3:14])

# select manually all columns by index required in vente_table
logement_columns = list(df.columns[14:])

# define dict of table_columns
table_dict = {"vente" : vente_columns,
              "logement" : logement_columns,
              "adresse" : adresse_columns
             }
for key, item in table_dict.items():
    print(key, ":")
    print(item, "\n")

# table w referenced_by u, v 
# => {w:[u, v]}
ref_dict = {"vente" : ["logement"], 
            "logement" : ["adresse"]
           }

vente :
['date_mutation', 'nature_mutation', 'valeur_fonciere'] 

logement :
['section', 'no_plan', 'no_volume', '1er_lot', 'surface_carrez_du_1er_lot', 'nombre_de_lots', 'code_type_local', 'type_local', 'surface_reelle_bati', 'nombre_pieces_principales'] 

adresse :
['no_voie', 'b/t/q', 'code_type_de_voie', 'type_de_voie', 'code_voie', 'voie', 'code_id_commune', 'code_postal', 'commune', 'code_departement', 'code_commune'] 



In [3]:
# tables_dict to save to tables.csv

def define_table_list(table_dict):
    to_csv = []
    
    for key, values in table_dict.items():
        for value in values:
            template = {"Table": key, 
                        "columns": value,
                        "data_type": df[value].dtypes
                       }
            to_csv.append(template)
    return to_csv

def define_reference_list(ref_dict):
    to_csv = []
    
    for key, values in ref_dict.items():
        for value in values:
            template = {"table": key, 
                        "referenced_by": value,
                       }
            to_csv.append(template)
    return to_csv

In [11]:
from csv_tools import dict_to_csv

# save table_dict to csv
to_csv = define_table_list(table_dict)
dict_to_csv(to_csv, "tables.csv")

# save ref_dict to csv
to_csv = define_reference_list(ref_dict)
dict_to_csv(to_csv, "references.csv")

'Dictionnary saved to csv successfully'

In [12]:
%run tables_generator.py

metadata_obj is online
Tables creation successful
Columns added to Tables successfully 

metadata_obj : 

Table : logement ; Column : id
Table : logement ; Column : section
Table : logement ; Column : no_plan
Table : logement ; Column : no_volume
Table : logement ; Column : 1er_lot
Table : logement ; Column : surface_carrez_du_1er_lot
Table : logement ; Column : nombre_de_lots
Table : logement ; Column : code_type_local
Table : logement ; Column : type_local
Table : logement ; Column : surface_reelle_bati
Table : logement ; Column : nombre_pieces_principales
Table : logement ; Column : adresse_id
Table : vente ; Column : id
Table : vente ; Column : date_mutation
Table : vente ; Column : nature_mutation
Table : vente ; Column : valeur_fonciere
Table : vente ; Column : logement_id
Table : adresse ; Column : id
Table : adresse ; Column : no_voie
Table : adresse ; Column : b/t/q
Table : adresse ; Column : code_type_de_voie
Table : adresse ; Column : type_de_voie
Table : adresse ; Column : 

In [18]:
from sqlalchemy import create_engine

# connect to immobilier_db
engine = create_engine('sqlite:///immobilier.db', echo = True)
connection = engine.connect()

In [19]:
# create all tables in metadata_obj
metadata_obj.create_all(engine)

2022-10-17 19:38:36,769 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-17 19:38:36,769 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("logement")
2022-10-17 19:38:36,770 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-17 19:38:36,770 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("logement")
2022-10-17 19:38:36,770 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-17 19:38:36,771 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("vente")
2022-10-17 19:38:36,771 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-17 19:38:36,771 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("vente")
2022-10-17 19:38:36,771 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-17 19:38:36,772 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("adresse")
2022-10-17 19:38:36,772 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-17 19:38:36,772 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("adresse")
2022-10-17 19:38:36,773 INFO sqlalchemy.engine.Engine [raw sql] ()
2

In [20]:
# flag condition for df.drop_duplicates() condition
# different logements can have the same adresse
flag_tables = ["adresse"]

# get sub_dataframe 
def get_sub_df(table_name):
    table_attributs = table_dict[table_name]
    sub_df = df.loc[:, table_attributs]
    
    # flag condition
    if table_name in flag_tables:
        sub_df = sub_df.drop_duplicates()
    else:
        pass
    return sub_df

# populate table from sub_dataframe
def populate_tables_from_df(table_dict):
    for table_name in table_dict.keys():
        # list of all insert statements
        ins = []
        # select Table object
        sql_table = metadata_obj.tables[table_name]
        
        # iterate for corresponding table’s sub_dataframe
        for i in range(len(get_sub_df(table_name))):
            # dict to save {column:row_value}
            ins_dict = {}
            # iterate for columns in sql_table
            for column in table_dict[table_name]:
                ins_dict[column] = df.loc[i, column]
            ins.append(ins_dict)
        # execute INSERT sql statement
        connection.execute(sql_table.insert(), ins)
        print("\n")
        
populate_tables_from_df(table_dict)

2022-10-17 19:38:40,046 INFO sqlalchemy.engine.Engine INSERT INTO vente (date_mutation, nature_mutation, valeur_fonciere) VALUES (?, ?, ?)
2022-10-17 19:38:40,047 INFO sqlalchemy.engine.Engine [generated in 0.09801s] (('2020-02-03 00:00:00.000000', 'Vente', 56000.0), ('2020-01-02 00:00:00.000000', 'Vente', 165000.0), ('2020-01-08 00:00:00.000000', 'Vente', 720000.0), ('2020-01-06 00:00:00.000000', 'Vente', 429250.0), ('2020-01-07 00:00:00.000000', 'Vente', 220900.0), ('2020-01-21 00:00:00.000000', 'Vente', 42000.0), ('2020-01-07 00:00:00.000000', 'Vente', 262000.0), ('2020-01-08 00:00:00.000000', 'Vente', 190000.0)  ... displaying 10 of 34169 total bound parameter sets ...  ('2020-06-25 00:00:00.000000', 'Vente', 725000.0), ('2020-06-11 00:00:00.000000', 'Vente', 433000.0))
2022-10-17 19:38:40,089 INFO sqlalchemy.engine.Engine COMMIT


2022-10-17 19:38:41,958 INFO sqlalchemy.engine.Engine INSERT INTO logement (section, no_plan, no_volume, "1er_lot", surface_carrez_du_1er_lot, nombre_de

In [21]:
connection.close()
engine.dispose()